In [1]:

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import validation_curve

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
mergedf = pd.read_csv('metadata_otu_5000_merged.csv', low_memory=False)
# mergedf = pd.read_csv('DB_CSV_merged_v1', low_memory=False)
mergedf = mergedf.drop("Unnamed: 0", axis=1)
mergedf.head(1)

,sample_name,age,age_in_years,age_units,agegroup,anonymized_name,body_habitat,body_product,body_site,collection_timestamp,...,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Prevotellaceae; g__Prevotella; s__.141,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__.173,k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__ACK-M1; g__; s__.5,k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__Actinomycetaceae; g__; s__.10,k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Pseudomonadales; f__Pseudomonadaceae; g__Pseudomonas; s__.64,k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Veillonellaceae; g__Dialister; s__.21,k__Bacteria; p__Bacteroidetes; c__Cytophagia; o__Cytophagales; f__Cytophagaceae; g__; s__.74,k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__Propionibacteriaceae; g__Propionibacterium; s__granulosum.2,k__Bacteria; p__Actinobacteria; c__Acidimicrobiia; o__Acidimicrobiales; f__; g__; s__.106,k__Bacteria; p__Proteobacteria; c__Alphaproteobacteria; o__Caulobacterales; f__Caulobacteraceae; g__Phenylobacterium; s__.14
0,797.02.HA.Forehead,1,1,years,Adolescent,FS_01,UBERON:skin,UBERON:sebum,UBERON:face,4/6/2010,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# mergedf.body_site.value_counts()
mergedf.shape

(896, 12541)

In [4]:
# simpledf = mergedf[["sample_name","sample_type","host_common_name",]]
gr=mergedf.loc[(mergedf!=0).any(axis=1)]
print(gr.shape)
gr

(896, 12541)


,sample_name,age,age_in_years,age_units,agegroup,anonymized_name,body_habitat,body_product,body_site,collection_timestamp,...,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Prevotellaceae; g__Prevotella; s__.141,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__.173,k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__ACK-M1; g__; s__.5,k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__Actinomycetaceae; g__; s__.10,k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Pseudomonadales; f__Pseudomonadaceae; g__Pseudomonas; s__.64,k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Veillonellaceae; g__Dialister; s__.21,k__Bacteria; p__Bacteroidetes; c__Cytophagia; o__Cytophagales; f__Cytophagaceae; g__; s__.74,k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__Propionibacteriaceae; g__Propionibacterium; s__granulosum.2,k__Bacteria; p__Actinobacteria; c__Acidimicrobiia; o__Acidimicrobiales; f__; g__; s__.106,k__Bacteria; p__Proteobacteria; c__Alphaproteobacteria; o__Caulobacterales; f__Caulobacteraceae; g__Phenylobacterium; s__.14
0,797.02.HA.Forehead,1,1,years,Adolescent,FS_01,UBERON:skin,UBERON:sebum,UBERON:face,4/6/2010,...,0,0,0,0,0,0,0,0,0,0
1,797.02.HA.LeftPalm,1,1,years,Adolescent,FS_01,UBERON:skin,UBERON:sebum,UBERON:skin of hand,4/6/2010,...,0,0,0,0,0,0,0,0,0,0
2,797.02.HA.Stool,1,1,years,Adolescent,FS_01,UBERON:feces,UBERON:feces,UBERON:feces,4/6/2010,...,1,96,0,0,0,0,0,0,0,0
3,797.02.HA.Tongue,1,1,years,Adolescent,FS_01,UBERON:oral cavity,UBERON:saliva,UBERON:tongue,4/6/2010,...,0,0,0,0,0,0,0,0,0,0
4,797.02.HB.Forehead,35,35,years,Adult,FS_02,UBERON:skin,UBERON:sebum,UBERON:face,4/6/2010,...,0,22,0,0,0,0,0,0,0,0
5,797.02.HB.RightPalm,35,35,years,Adult,FS_02,UBERON:skin,UBERON:sebum,UBERON:skin of hand,4/6/2010,...,0,1,0,0,0,0,0,5,0,0
6,797.02.HB.Stool,35,35,years,Adult,FS_02,UBERON:feces,UBERON:feces,UBERON:feces,4/6/2010,...,1,88,0,0,0,0,0,0,0,0
7,797.02.HB.Tongue,35,35,years,Adult,FS_02,UBERON:oral cavity,UBERON:saliva,UBERON:tongue,4/6/2010,...,0,0,0,0,0,0,0,0,0,0
8,797.02.HC.Forehead,3,3,years,Adolescent,FS_03,UBERON:skin,UBERON:sebum,UBERON:face,4/6/2010,...,0,0,0,0,0,0,0,0,0,0
9,797.02.HC.Stool,3,3,years,Adolescent,FS_03,UBERON:feces,UBERON:feces,UBERON:feces,4/6/2010,...,0,14,0,0,0,0,0,0,0,0


In [7]:
gr = mergedf.groupby("familyID").sum()
gr

,dna_extracted,elevation,family,host_taxid,latitude,longitude,physical_specimen_remaining,public,qiita_study_id,taxon_id,...,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Prevotellaceae; g__Prevotella; s__.141,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__.173,k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__ACK-M1; g__; s__.5,k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__Actinomycetaceae; g__; s__.10,k__Bacteria; p__Proteobacteria; c__Gammaproteobacteria; o__Pseudomonadales; f__Pseudomonadaceae; g__Pseudomonas; s__.64,k__Bacteria; p__Firmicutes; c__Clostridia; o__Clostridiales; f__Veillonellaceae; g__Dialister; s__.21,k__Bacteria; p__Bacteroidetes; c__Cytophagia; o__Cytophagales; f__Cytophagaceae; g__; s__.74,k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__Propionibacteriaceae; g__Propionibacterium; s__granulosum.2,k__Bacteria; p__Actinobacteria; c__Acidimicrobiia; o__Acidimicrobiales; f__; g__; s__.106,k__Bacteria; p__Proteobacteria; c__Alphaproteobacteria; o__Caulobacterales; f__Caulobacteraceae; g__Phenylobacterium; s__.14
familyID,,,,,,,,,,,,,,,,,,,,,
family#1020,5.0,8120.5,5100,48030,200.05,-526.35,0.0,5.0,3985,2474561,...,0,0,0,0,0,0,0,115,0,0
family#1053,4.0,6496.4,4212,38424,160.04,-421.08,0.0,4.0,3188,2066391,...,0,0,0,0,1,0,0,63,0,0
family#1075,3.0,4872.3,3225,28818,120.03,-315.81,0.0,3.0,2391,1395251,...,0,618,0,0,0,0,0,38,0,0
family#1148,4.0,6496.4,4592,38424,160.04,-421.08,0.0,4.0,3188,1934906,...,0,75,0,0,0,0,0,3,0,0
family#1191,5.0,8120.5,5955,48030,200.05,-526.35,0.0,5.0,3985,2474561,...,0,994,0,0,0,0,0,101,0,0
family#12,15.0,24361.5,180,144090,600.15,-1579.05,0.0,15.0,11955,7331454,...,0,3,0,0,0,0,0,36,0,0
family#1204,4.0,6496.4,4816,38424,160.04,-421.08,0.0,4.0,3188,1934906,...,0,234,0,0,0,0,0,13,0,0
family#1239,3.0,4872.3,3717,28818,120.03,-315.81,0.0,3.0,2391,1395251,...,0,1,0,0,0,0,0,1,0,0
family#1253,3.0,4872.3,3759,28818,120.03,-315.81,0.0,3.0,2391,1395251,...,0,46,0,0,0,0,0,1,0,0


# Filtering Train and Test Datasets

In [ ]:
#FILTER 1
have_dog = "only_non_dog_owners"

#FILTER 2
sample_type = "skin"
# sample_type = "all"
# #FILTER 3
# # not_sample_type = "stool"

# # #FILTER 4
human_role = "Partner"
# human_role = "all"
# #FILTER 5
# not_human_role = "Offspring"

#FILTER 6
# not_familyID = 63

# SPLITTING TRAIN and TEST (test = dogs, train = humans)

In [ ]:
human_data_train = mergedf[mergedf.host_common_name =="human"]

dog_data_test = mergedf[mergedf.host_common_name =="dog"]
dog_data_test.head(1)

In [ ]:
# #testing dog forehead to human right palm hypothesis

# human_data_train = human_data_train[human_data_train.body_site =="UBERON:skin of hand"]
# print(human_data_train.shape)
# dog_data_test = dog_data_test[dog_data_test.body_site =="UBERON:face"]
# print(dog_data_test.shape)

In [ ]:
# Applying filters above
try:
    if have_dog == "only_dog_owners":
        human_data_train = human_data_train[human_data_train.have_dog =="yes"]
    
    elif have_dog == "only_non_dog_owners":
        human_data_train = human_data_train[human_data_train.have_dog =="no"]
    elif have_dog == "all":
        human_data_train = human_data_train
    else:
        human_data_train = human_data_train
except:
    print("no filter here 1")
print(human_data_train.shape)

try:    
    if sample_type!= "all":
        human_data_train = human_data_train[human_data_train.sample_type ==sample_type]
        dog_data_test = dog_data_test[dog_data_test.sample_type ==sample_type]
    elif sample_type == "all":
        human_data_train = human_data_train
        dog_data_test = dog_data_test
    else:
        human_data_train = human_data_train
        dog_data_test = dog_data_test
except:
    print("no filter here 2")
print(human_data_train.shape)  

# try:    
#     if not_sample_type:
#         human_data_train = human_data_train[human_data_train.sample_type !=not_sample_type]
#         dog_data_test = dog_data_test[dog_data_test.sample_type !=not_sample_type]
#     elif sample_type == "all":
#         human_data_train = human_data_train
#         dog_data_test = dog_data_test
# except:
#     print("no filter here 3")
    
try:    
    if human_role != "all":
        human_data_train = human_data_train[human_data_train.role ==human_role]
    elif human_role == "all":
        human_data_train = human_data_train
    else:
        human_data_train = human_data_train
except:
    print("no filter here 4")
print(human_data_train.shape)    
# try:
#     if not_human_role:
#         human_data_train = human_data_train[human_data_train.role !=not_human_role]    
# except:
#     print("no filter here 5")
    
# try:    
#     if not_familyID:
#         human_data_train = human_data_train[human_data_train.role !=not_familyID]
#         dog_data_test = dog_data_test[dog_data_test.role !=not_familyID]  
# except:
#     print("no filter here 6")

# Train/test split

In [ ]:
X_train = human_data_train.iloc[:, -1035:-1]
y_train = human_data_train["familyID"].values.reshape(-1, 1)

y_test = dog_data_test["familyID"].values.reshape(-1, 1)
X_test = dog_data_test.iloc[:, -1035:-1]

# Hyperparameter Selection

In [ ]:
n_estimators = 1000

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=n_estimators)

# Non-Scaled Model Fitting

In [ ]:
#from non-scaled data
rf = rf.fit(X_train, y_train.ravel())
rf.score(X_test, y_test) #geni impurity coefficient

print(f'Test Score: {rf.score(X_test, y_test)}')
print(f'Train Score: {rf.score(X_train, y_train)}')

# Scaled X Model Fitting

In [ ]:
# from sklearn.preprocessing import StandardScaler
# X_scaler = StandardScaler().fit(X_train)

# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# from scaled data
# rf_scaled = RandomForestClassifier(n_estimators=n_estimators)
# rf_scaled = rf_scaled.fit(X_train_scaled, y_train.ravel())
# rf_scaled.score(X_test_scaled, y_test) #geni impurity coefficient

# print(f'Test Score: {rf_scaled.score(X_test_scaled, y_test)}')
# print(f'Train Score: {rf_scaled.score(X_train_scaled, y_train)}')

# Evaluate most-guessed family for each dog (7 collection site/predictions per animal) ==> 1 prediction

In [ ]:
y_pred = rf.predict(X_test)

outputdf = pd.DataFrame({"SampleName": dog_data_test["sample_name"],"Anonymized_Name":dog_data_test["anonymized_name"],"Prediction": y_pred, "Actual": y_test.ravel()}).reset_index(drop=True)
print(outputdf.shape)
outputdf.head()

In [ ]:
sumOutput = outputdf.groupby(["Anonymized_Name","Actual","Prediction"]).count()
sumOutput.reset_index()
sumOutput.head()

In [ ]:
# sumOutput.groupby(["Anonymized_Name"])['SampleName'].max()
idx = sumOutput.groupby(["Anonymized_Name"])['SampleName'].transform(max) == sumOutput['SampleName']
maxOutput = sumOutput[idx]
maxOutput = maxOutput.reset_index()
print(maxOutput.shape)
# maxOutput

In [ ]:
maxOutput_noDup = maxOutput.drop_duplicates(subset=['Anonymized_Name'], keep="first")
print(maxOutput_noDup.shape)
# maxOutput_noDup

In [ ]:
maxOutput_C = maxOutput_noDup[maxOutput_noDup.Actual == maxOutput_noDup.Prediction]

print(maxOutput_C.shape)
# maxOutput_C

In [ ]:
maxOutput_I = maxOutput_noDup[maxOutput_noDup.Actual != maxOutput_noDup.Prediction]

print(maxOutput_I.shape)
# maxOutput_I

In [ ]:
correct_total = maxOutput_C["Anonymized_Name"].count()
incorrect_total = maxOutput_I["Anonymized_Name"].count()

accuracy_maxed_pred = correct_total/(correct_total + incorrect_total)
print(f' Accuracy of most-predicted (dog) family: {round(accuracy_maxed_pred,3)}')